# Load multiband raster data via stream API

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

import geoengine as ge

## Initialize Geo Engine

In [2]:
ge.initialize("http://localhost:3030/api")

## Define workflow of a stacked raster of ndvi and land cover

In [3]:
workflow = ge.register_workflow({
                "type": "Raster",
                "operator": {
                    "type": "RasterStacker",
                    "params": {},
                    "sources": {
                        "rasters": [
                            {
                                "type": "GdalSource",
                                "params": {
                                    "data": "ndvi"
                                }
                            }, {
                                "type": "GdalSource",
                                "params": {
                                    "data": "land_cover"
                                }
                            }
                        ]
                    }
                }
            })
workflow

3fb85bcc-4b88-5c39-a48e-ac6833d69a8e

## Define a query rectangle

In [4]:
time_start = datetime.strptime('2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")
time_end = datetime.strptime('2014-06-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")
bbox = ge.QueryRectangle(
    ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
    ge.TimeInterval(time_start, time_end),
    ge.SpatialResolution(0.5, 0.5),
)

# Query single tiles, merge them locally and output them as a single `DataArray`

In [5]:
# setting `clip_to_query_rectangle=True` may output a warning, but this is a false positive from the underlying library
warnings.simplefilter(action='ignore', category=FutureWarning)

array = await workflow.raster_stream_into_xarray(bbox, bands=[0, 1])

array.isel(band=0).plot(col="time")
plt.suptitle("Band 0 (NDVI) over time", y=1.05)
plt.show()

array.isel(band=1).plot(col="time")
plt.suptitle("Band 1 (Land cover) over time", y=1.05)
plt.show()


MergeError: combine_attrs='no_conflicts', but some values are not the same. Merging {'tile_idx_y': -1, 'tile_idx_x': -1} with {'tile_idx_y': -1, 'tile_idx_x': 0}